In [1]:
# Import packages
import pandas as pd
import re

# Set  number format
# np.set_printoptions(precision=6, suppress=True)
pd.options.display.float_format = "{:,.2f}".format

#### INDUSTRY x INDUSTRY Table

In [2]:
# ---------------------------------------------------------
# SPECIFY PATH
# ---------------------------------------------------------
dir_path = "../data/exiobase/_ixi/IOT_2019_ixi/"


# ---------------------------------------------------------
# IMPORT MATRICES
# ---------------------------------------------------------
# Import A, Y, F and F_Y (from SATELLITE)
# ! 'A' is a large datasets, mind load time
Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])

# Get X reference from Exiobase
x_exio = pd.read_csv(
    f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1]
)  # ! header = 0, not [0,1]

# ! we use the satellite folder this time because we will look at uncharacterized stressors (CO2 emission vs GWP100)
F = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)

# ---------------------------------------------------------
# GET LABEL
# ---------------------------------------------------------
# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()


In [5]:
MASK = sector_labels.str.contains("gas", flags=re.IGNORECASE)
sector_labels[MASK]


Index(['Manure treatment (biogas), storage and land application',
       'Extraction of natural gas and services related to natural gas extraction, excluding surveying',
       'Extraction, liquefaction, and regasification of other petroleum and gaseous materials',
       'Production of electricity by gas',
       'Manufacture of gas; distribution of gaseous fuels through mains',
       'Biogasification of food waste, incl. land application',
       'Biogasification of paper, incl. land application',
       'Biogasification of sewage slugde, incl. land application'],
      dtype='object', name='sector')

In [6]:
EU27 = region_labels[:27]
RU = ["RU"]
biogas = ["Biogas"]
nat_gas = [
    "Extraction of natural gas and services related to natural gas extraction, excluding surveying",
    "Manufacture of gas; distribution of gaseous fuels through mains",
    "Extraction, liquefaction, and regasification of other petroleum and gaseous materials",
    "Transport via pipelines",
]

EU27_nat_gas = pd.MultiIndex.from_product(iterables=[EU27, nat_gas])
RU_nat_gas = pd.MultiIndex.from_product(iterables=[RU, nat_gas])

In [7]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY)
Z.loc[RU_nat_gas].groupby(level=0, axis=1).sum().T


region                                                                                            RU   
sector Extraction of natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                                 
AT                                                   0.50                                             \
AU                                                   0.23                                              
BE                                                  33.20                                              
BG                                                   7.01                                              
BR                                                  12.54                                              
CA                                                   4.25                                              
CH                                                  17.82                                              
CN                                                 309.93                                              
CY                                                   0.47                                              
CZ                                               1,298.47                                              
DE                                                  63.11                                              
DK                                                  26.12                                              
EE                                                 116.50                                              
ES                                                  19.51                                              
FI                                                  29.89                                              
FR                                                  39.29                                              
GB                                                  56.92                                              
GR                                                  58.01                                              
HR                                                   1.11                                              
HU                                               1,076.45                                              
ID                                                   0.60                                              
IE                                                  32.26                                              
IN                                                  12.27                                              
IT                                               6,378.79                                              
JP                                                  24.97                                              
KR                                                  95.96                                              
LT                                                 252.05                                              
LU                                                   0.64                                              
LV                                                 238.59                                              
MT                                                  46.70                                              
MX                                                   1.05                                              
NL                                                  11.00                                              
NO                                                   8.94                                              
PL                                                  53.02                                              
PT                                                   1.71                                              
RO                                              

In [8]:
# How much does each EU country relies on DIRECT RU gas sectors (FINAL DEMAND)
Y.loc[RU_nat_gas].groupby(level=0, axis=1).sum().T

region                                                                                            RU   
sector Extraction of natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                                 
AT                                                   0.31                                             \
AU                                                   0.17                                              
BE                                                   9.26                                              
BG                                                 562.06                                              
BR                                                   0.92                                              
CA                                                   2.25                                              
CH                                                   0.95                                              
CN                                                   6.89                                              
CY                                                   0.26                                              
CZ                                                   0.89                                              
DE                                                  36.89                                              
DK                                                   6.85                                              
EE                                                   3.37                                              
ES                                                   2.89                                              
FI                                                   2.20                                              
FR                                                  18.35                                              
GB                                                  31.19                                              
GR                                                 722.38                                              
HR                                                   0.51                                              
HU                                                  26.79                                              
ID                                                   0.11                                              
IE                                                   4.53                                              
IN                                                   2.82                                              
IT                                                  17.18                                              
JP                                                   3.98                                              
KR                                                  18.87                                              
LT                                                   1.66                                              
LU                                                   0.77                                              
LV                                                   9.62                                              
MT                                                  14.97                                              
MX                                                   0.25                                              
NL                                                   9.58                                              
NO                                                   3.42                                              
PL                                                   8.22                                              
PT                                                   0.13                                              
RO                                              

In [9]:
# THE CASE OF GERMANY (INTER INDUSTRY)
print(Z.loc["RU", "DE"].sum(axis=1).to_string())

sector
Cultivation of paddy rice                                                                                                                  0.00
Cultivation of wheat                                                                                                                       8.43
Cultivation of cereal grains nec                                                                                                          13.37
Cultivation of vegetables, fruit, nuts                                                                                                     6.50
Cultivation of oil seeds                                                                                                                  16.26
Cultivation of sugar cane, sugar beet                                                                                                      0.65
Cultivation of plant-based fibers                                                                                                

In [10]:
# THE CASE OF GERMANY (FINAL DEMAND)
print(Y.loc["RU", "DE"].sum(axis=1).to_string())

sector
Cultivation of paddy rice                                                                                                                0.00
Cultivation of wheat                                                                                                                     3.18
Cultivation of cereal grains nec                                                                                                         3.86
Cultivation of vegetables, fruit, nuts                                                                                                  14.56
Cultivation of oil seeds                                                                                                                 9.40
Cultivation of sugar cane, sugar beet                                                                                                    0.16
Cultivation of plant-based fibers                                                                                                        0.00

#### PRODUCT x PRODUCT Table

In [ ]:
# ---------------------------------------------------------
# SPECIFY PATH
dir_path = "../data/exiobase/_pxp/IOT_2019_pxp/"


# ---------------------------------------------------------
# IMPORT MATRICES

# Import A, Y, F and F_Y (from SATELLITE)
# ! 'A' is a large datasets, mind load time
Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])

# Get X reference from Exiobase
x_exio = pd.read_csv(
    f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1]
)  # ! header = 0, not [0,1]

# ! we use the satellite folder this time because we will look at uncharacterized stressors (CO2 emission vs GWP100)
F = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)

# ---------------------------------------------------------
# GET LABEL

# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()


In [107]:
MASK = sector_labels.str.contains("gas", flags=re.IGNORECASE)
sector_labels[MASK]


In [85]:
EU27 = region_labels[:27]
RU = ["RU"]
biogas = ["Biogas"]
nat_gas = [
    "Natural gas and services related to natural gas extraction, excluding surveying",
    "Distribution services of gaseous fuels through mains",
    "Natural Gas Liquids",
    "Transportation services via pipelines",
]

EU27_nat_gas = pd.MultiIndex.from_product(iterables=[EU27, nat_gas])
RU_nat_gas = pd.MultiIndex.from_product(iterables=[RU, nat_gas])

In [86]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY)
Z.loc[RU_nat_gas].groupby(level=0, axis=1).sum().T


region                                                                              RU   
sector Natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                   
AT                                                   0.00                               \
AU                                                   0.00                                
BE                                                   0.00                                
BG                                                   3.72                                
BR                                                   0.00                                
CA                                                   0.00                                
CH                                                  14.34                                
CN                                                   0.00                                
CY                                                   0.00                                
CZ                                               1,452.27                                
DE                                                   0.00                                
DK                                                   0.00                                
EE                                                 111.18                                
ES                                                   0.00                                
FI                                                   0.00                                
FR                                                   0.00                                
GB                                                   0.00                                
GR                                                   3.49                                
HR                                                   0.00                                
HU                                               1,194.48                                
ID                                                   0.00                                
IE                                                   0.00                                
IN                                                   0.00                                
IT                                               7,110.88                                
JP                                                   0.00                                
KR                                                   0.01                                
LT                                                 278.04                                
LU                                                   0.00                                
LV                                                 251.12                                
MT                                                   0.00                                
MX                                                   0.00                                
NL                                                   0.00                                
NO                                                   0.00                                
PL                                                   0.00                                
PT                                                   0.00                                
RO                                                   0.54                                
RU                                                  52.94                                
SE                                                   0.00                                
SI                                                  22.48                                
SK                                                   3.48                                
TR                                                   0.00                                
TW                                      

In [87]:
# How much does each EU country relies on DIRECT RU gas sectors (FINAL DEMAND)
Y.loc[RU_nat_gas].groupby(level=0, axis=1).sum().T


region                                                                              RU   
sector Natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                   
AT                                                   0.00                               \
AU                                                   0.00                                
BE                                                   0.00                                
BG                                                 628.19                                
BR                                                   0.00                                
CA                                                   0.00                                
CH                                                   0.00                                
CN                                                   0.00                                
CY                                                   0.00                                
CZ                                                   0.45                                
DE                                                   0.00                                
DK                                                   0.00                                
EE                                                   0.11                                
ES                                                   0.00                                
FI                                                   0.00                                
FR                                                   0.00                                
GB                                                   0.00                                
GR                                                 784.91                                
HR                                                   0.00                                
HU                                                  28.56                                
ID                                                   0.00                                
IE                                                   0.00                                
IN                                                   0.00                                
IT                                                  13.96                                
JP                                                   0.00                                
KR                                                   0.00                                
LT                                                   0.00                                
LU                                                   0.00                                
LV                                                   2.75                                
MT                                                   0.00                                
MX                                                   0.00                                
NL                                                   0.00                                
NO                                                   0.00                                
PL                                                   0.00                                
PT                                                   0.00                                
RO                                                 340.37                                
RU                                               2,883.01                                
SE                                                   0.00                                
SI                                                  91.37                                
SK                                               1,484.12                                
TR                                                   0.00                                
TW                                      

In [104]:
# THE CASE OF GERMANY (INTER INDUSTRY)
print(Z.loc["RU", "DE"].sum(axis=1).to_string())

sector
Paddy rice                                                                                                                                    0.00
Wheat                                                                                                                                         1.65
Cereal grains nec                                                                                                                            10.64
Vegetables, fruit, nuts                                                                                                                       2.75
Oil seeds                                                                                                                                    14.03
Sugar cane, sugar beet                                                                                                                        0.00
Plant-based fibers                                                                                             

In [105]:
# THE CASE OF GERMANY (FINAL DEMAND)
print(Y.loc["RU", "DE"].sum(axis=1).to_string())

sector
Paddy rice                                                                                                                                  0.00
Wheat                                                                                                                                       2.79
Cereal grains nec                                                                                                                           3.51
Vegetables, fruit, nuts                                                                                                                    13.29
Oil seeds                                                                                                                                   9.22
Sugar cane, sugar beet                                                                                                                      0.00
Plant-based fibers                                                                                                         